Indeed Job Scraper and Dataizer
By Vivek
3/22/16

Code to scrape indeed for their data on job searches.  What will it become?

My plan is first to score and rank jobs.  Then maybe a framework for organizing what jobs match me by skillset.  Or finding unique words that describe jobs.

In [23]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import collocations
import nltk
import numpy as np
import pandas as pd

In [2]:
#http://docs.python-requests.org/en/master/

In [3]:
# r = requests.get('http://api.indeed.com/ads/apisearch?publisher=8538934413867228&q=data+scientist&l=san+jose%2C+ca&sort=&radius=10&st=&jt=&start=37&limit=25&fromage=&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2')

In [4]:
def getTextForUrl(job_url):
    r = requests.get(job_url)
    soup = BeautifulSoup(r.text,'html.parser')
    text=soup.find(id="job_summary").get_text().replace('\n',' ')
    return text

In [5]:
def search_string(q,l,radius,start,age):
    st=''
    jt=''
    limit=25
    search='http://api.indeed.com/ads/apisearch?publisher=8538934413867228&q=%s&l=%s&sort=&radius=%d&st=%s&jt=%s&start=%d&limit=%d&fromage=%d&filter=1&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Chromazord&v=2' % (q,l,radius,st,jt,start,limit,age)
    return search

In [6]:
origin = []
maxdist = []
url = []
jobtitle = []
company = []
city = []
state = []
country = []
source = []
date = []
snippet = []
latitude = []
longitude = []
jobkey = []
sponsored = []
expired = []
snippet = []
indeedApply = []
#Derived
description = []

In [7]:
q='data+scientist'
l='san+jose%2C+ca'
radius=20
start=0
age=25
search = search_string(q,l,radius,start,age)
req=requests.get(search)
root = ET.fromstring(req.text)
total_results = int(root.find('totalresults').text)
#results = root.find('results').findall('result')

In [8]:
total_results

347

In [9]:
q='data+scientist'
l='san+jose%2C+ca'
radius=20
start=0
age=25
for start in range(0, total_results,25):
    search = search_string(q,l,radius,start,age)
    req=requests.get(search)
    root = ET.fromstring(req.text)
    results = root.find('results').findall('result')
    for r in results:
        origin.append(l)
        maxdist.append(radius)
        url.append(r.find('url').text)
        company.append(r.find('company').text)
        city.append(r.find('city').text)
        state.append(r.find('state').text)
        jobtitle.append(r.find('jobtitle').text)
        country.append(r.find('country').text)
        source.append(r.find('source').text)
        date.append(r.find('date').text)
        jobkey.append(r.find('jobkey').text)
        snippet.append(r.find('snippet').text)
        latitude.append(r.find('latitude').text)
        longitude.append(r.find('longitude').text)
        sponsored.append(r.find('sponsored').text)
        expired.append(r.find('expired').text)
        indeedApply.append(r.find('indeedApply').text)        

In [10]:
counter = 1
for each in url:
    description.append(getTextForUrl(each))
    print(counter)
    counter = counter + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
type(description)
len(description)

346

In [12]:
#http://pandas.pydata.org/pandas-docs/stable/dsintro.html
df = pd.DataFrame({'jobtitle' : jobtitle, "company" : company, 'city' : city, "state" : state, 
                   "country" : country, "source" : source, "date" : date, 
                   "jobkey" : jobkey, "description" : description, "snippet" : snippet,
                   "latitude" : latitude, "longitude" : longitude, "sponsored" : sponsored,
                   "expired" : expired, "indeedApply" : indeedApply, 'origin' : origin})

In [13]:
len(description)

346

In [14]:
num = 1
results[num].find('date').text

'Thu, 07 Apr 2016 00:06:54 GMT'

In [15]:
import datetime
today = datetime.date.today().isoformat()

In [16]:
df.to_csv(path_or_buf = today + " sanjosejobs.csv")

In [17]:
df2 = pd.read_csv("sanjosejobs.csv", index_col = 0)

In [18]:
df = df2

In [19]:
from collections import Counter

In [20]:
type(df.description[2])

str

In [21]:
first = df.description[1]

In [22]:
first[0:100]

'Position : Data Scientist – ModellerLocation : San Jose, CADuration: Fulltime / Permanent1)Data Scie'

In [24]:
nltk.word_tokenize(first)

['Position',
 ':',
 'Data',
 'Scientist',
 '–',
 'ModellerLocation',
 ':',
 'San',
 'Jose',
 ',',
 'CADuration',
 ':',
 'Fulltime',
 '/',
 'Permanent1',
 ')',
 'Data',
 'Scientist',
 '–*',
 '*ModellerFluent',
 'in',
 'Python.HADOOP',
 ',',
 'Java',
 ',',
 'Python',
 ',',
 'HIVE/PIG',
 'for',
 'extracting',
 'and',
 'parsing',
 'data.Visualization',
 'Skills',
 '-',
 'JavaScript',
 '(',
 'especially',
 'D3.js',
 ')',
 '.The',
 'person',
 'should',
 'have',
 'some',
 'knowledge',
 'of',
 'Stats',
 '&',
 'Machine',
 'Learning.Position',
 ':',
 'Data',
 'Scientist',
 '–',
 'Software',
 'Engineer',
 '(',
 'SWE',
 ')',
 'ModellerLocation',
 ':',
 'San',
 'Jose',
 ',',
 'CADuration',
 ':',
 'Fulltime',
 '/',
 'Permanent2',
 ')',
 'Data',
 'Scientist',
 '–',
 'Software',
 'Engineer',
 '(',
 'SWE',
 ')',
 'Statistical/data',
 'modeling/data',
 'mining',
 'and',
 'machine',
 'learningProficiency',
 'in',
 'SQL',
 'and',
 'dealing',
 'with',
 'petabytes',
 'of',
 'log',
 'files',
 'and',
 'other'

In [25]:
subject_word_bag = df.description.apply(lambda t: t.lower() + " ").sum()

In [26]:
subject_word_bag

'job description  the medical sieve group in cognitive computing foundations department at ibm research - almaden is looking for candidates in biomedical and clinical informatics for a new long term research project combining clinical and imaging data for enhancing clinical decision making in radiology, cardiology and other modalities. this is a position accepting applicants with bachelors or masters in electrical engineering/computer science or related field. candidates with background in medical text analysis, data management, clinical knowledge and reasoning, medical imaging analysis and analysis of large textual collections is desirable. knowledge and experience in clinical terminologies and coding systems such as umls is also preferred. applicant may have to travel to a client site for extraction, collection and gathering of data for analysis and machine learning. applicantswill be working with research scientists assisting them in data collection,data management, annotations, med

In [27]:
df.description.apply(lambda t: t.lower() + " ")

0     job description  the medical sieve group in co...
1     position : data scientist – modellerlocation :...
2     description intuit quickbooks is on a mission ...
3     (menlo park, ca) careers at oculus a facebook ...
4     data scientist with pythonlocation: cupertino ...
5     requisition no.: 121517br  subsidiary: ebay ma...
6     requisition no.: 29374br  subsidiary: paypal  ...
7     requisition no.: 119481br  subsidiary: ebay ma...
8     position: data scientist / big data engineer –...
9     discovering the perfect course is the first an...
10    requisition id: 105756 work area: software-des...
11    additional location(s) or information: job cat...
12    about retail solutions, inc.: retail solutions...
13    the bioinformatics, data analysis and statisti...
14    the role  play a central role in designing and...
15    requisition no.: 29268br  subsidiary: paypal  ...
16    as a leading fortune 500 technology & security...
17    your goal – to improve the education proce

In [28]:
len(subject_word_bag)

61792

In [29]:
Counter(subject_word_bag.split()).most_common()[:80]

[('and', 466),
 ('the', 250),
 ('to', 229),
 ('of', 210),
 ('in', 172),
 ('data', 159),
 ('a', 144),
 ('with', 128),
 ('is', 72),
 ('for', 72),
 ('or', 70),
 ('experience', 63),
 ('on', 52),
 ('are', 50),
 ('machine', 48),
 ('our', 46),
 ('as', 42),
 ('you', 41),
 ('team', 41),
 ('will', 41),
 ('that', 40),
 ('be', 38),
 ('product', 34),
 ('strong', 33),
 ('work', 33),
 ('at', 33),
 ('learning', 32),
 ('business', 28),
 ('solutions', 27),
 ('we', 25),
 ('software', 25),
 ('new', 24),
 ('more', 22),
 ('research', 22),
 ('this', 22),
 ('an', 22),
 ('knowledge', 20),
 ('analysis', 20),
 ('statistical', 20),
 ('working', 20),
 ('scientist', 20),
 ('skills', 20),
 ('develop', 20),
 ('other', 20),
 ('building', 19),
 ('job', 19),
 ('engineering', 19),
 ('computer', 18),
 ('have', 18),
 ('by', 18),
 ('using', 18),
 ('from', 18),
 ('technical', 18),
 ('years', 18),
 ('development', 18),
 ('ability', 18),
 ('your', 17),
 ('large', 17),
 ('&', 17),
 ('into', 17),
 ('one', 16),
 ('background', 15

In [30]:
stops = [word for word in stopwords.words('english')] + ['re:', 'fwd:', '-', '•']

In [31]:
#nltk.download()

In [32]:
subject_words = [word for word in subject_word_bag.split() if word.lower() not in stops]
Counter(subject_words).most_common()[:100]

[('data', 159),
 ('experience', 63),
 ('machine', 48),
 ('team', 41),
 ('product', 34),
 ('strong', 33),
 ('work', 33),
 ('learning', 32),
 ('business', 28),
 ('solutions', 27),
 ('software', 25),
 ('new', 24),
 ('research', 22),
 ('knowledge', 20),
 ('analysis', 20),
 ('statistical', 20),
 ('working', 20),
 ('scientist', 20),
 ('skills', 20),
 ('develop', 20),
 ('building', 19),
 ('job', 19),
 ('engineering', 19),
 ('computer', 18),
 ('using', 18),
 ('technical', 18),
 ('years', 18),
 ('development', 18),
 ('ability', 18),
 ('large', 17),
 ('&', 17),
 ('one', 16),
 ('background', 15),
 ('big', 15),
 ('responsibilities:', 15),
 ('analytics', 15),
 ('teams', 15),
 ('products', 15),
 ('science', 15),
 ('highly', 14),
 ('complex', 14),
 ('help', 14),
 ('design', 13),
 ('quality', 13),
 ('security', 13),
 ('systems', 13),
 ('applications', 13),
 ('communication', 13),
 ('management', 13),
 ('related', 13),
 ('learning,', 13),
 ('retail', 12),
 ('candidate', 12),
 ('techniques', 12),
 ('und

In [33]:
    #Create the pandas dataframe
    df = pd.DataFrame({'user_id' : user_id, "visits" : visits, 'total_purchases' : total_purchases, "time_max" : time_max, "time_min" : time_min, "time_diff" : time_diff})
    # In[10]:
    #Convert panadas elements to the proper types.  Put in columns for normalized variables.
    df[['total_purchases']]=df[['total_purchases']].astype(float)
    df[['user_id']]=df[['user_id']].astype(object)
    # In[11]:
    df['norm_max_time']=(df['time_max']-df['time_max'].mean())/(df['time_max'].max()-df['time_max'].min())
    # In[12]:
    df['norm_visits'] = (df['visits']-df['visits'].mean())/(df['visits'].max()-df['visits'].min())
    # In[13]:
    df['norm_total_purchases'] = (df['total_purchases']-df['total_purchases'].mean())/(df['total_purchases'].max()-df['total_purchases'].min())
    # In[17]:
    #class sklearn.cluster.KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances=True, verbose=0, random_state=None, copy_x=True, n_jobs=1)
    df[[6,7,8]].dtypes
    # In[18]:
    kmeans = KMeans(init ='k-means++', n_clusters = 4, n_init=10)
    #kmeans.fit(df)
    kmeans.fit(df[[6,7,8]])

NameError: name 'user_id' is not defined

In [34]:
results[0].

SyntaxError: invalid syntax (<ipython-input-34-ec6c88654d29>, line 1)

In [35]:
 r = requests.get(url[2].text)

AttributeError: 'str' object has no attribute 'text'

In [36]:
url[2]

'http://www.indeed.com/viewjob?jk=d73f090a45bd5232&qd=VJkvx8ARkUg4BTzBiPO74cOFQXA2SP39qeq0wN3mseLtI7ek5yI-2YCaRF90c_BlcRasNhazbz6_Wn8NDNbc7Ezd4EiRwSt1SnrSMsGpDG61ejuS5zYC3b4ikRuO41LG&indpubnum=8538934413867228&atk=1agvljgctb050fok'

type(root)

In [37]:
page = requests.get(url[2].text)
tree = html.fromstring(page.content)

AttributeError: 'str' object has no attribute 'text'

In [38]:
(We need to use page.content rather than page.text because html.fromstring implicitly expects bytes as input.)

SyntaxError: invalid syntax (<ipython-input-38-f0b77a89f2de>, line 1)

In [39]:
tree

NameError: name 'tree' is not defined

In [40]:
<div title="buyer-name">Carson Busses</div>
<span class="item-price">$29.95</span>
Knowing this we can create the correct XPath query and use the lxml xpath function like this:

#This will create a list of buyers:
buyers = tree.xpath('//div[@title="buyer-name"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="item-price"]/text()')

SyntaxError: invalid syntax (<ipython-input-40-b854f42b6b8d>, line 1)

In [41]:
#<span id="job_summary" class="summary">
summary = tree.xpath('//span[@id="job_summary"]/text()')

NameError: name 'tree' is not defined

In [42]:
print('Summary: ', summary)

NameError: name 'summary' is not defined

In [43]:
for element in tree.iter():
...     print("%s - %s" % (element.tag, element.text))

NameError: name 'tree' is not defined

In [44]:
m = re.search('\w+class="summary">(\d+)</span>', r.text)

NameError: name 're' is not defined

In [45]:
m = re.search(r'class="summary\">(.+)\<\/span>', r.text,re.DOTALL)

NameError: name 're' is not defined

In [46]:
m = re.search(r'</span>', r.text)
m

NameError: name 're' is not defined

In [47]:
r.text

'\n                    '

In [48]:
m = re.search(r'class="summary\">(.+)/span>', r.text,re.DOTALL)

NameError: name 're' is not defined

In [49]:
m

NameError: name 'm' is not defined

In [50]:
m.group()

NameError: name 'm' is not defined

In [51]:
m = re.search(r'job_summary([^<]+)<', r.text,re.DOTALL)

NameError: name 're' is not defined

m.group()

In [52]:
m = re.search(r'<title[^>]*>([^<]+)</title>',r.text)

NameError: name 're' is not defined

In [53]:
from bs4 import BeautifulSoup

In [54]:
print(soup.prettify())

NameError: name 'soup' is not defined

In [55]:
r = requests.get(url[0].text)
soup = BeautifulSoup(r.text,'html.parser')
text=soup.find(id="job_summary").get_text().replace('\n',' ')
text

AttributeError: 'str' object has no attribute 'text'

In [ ]:
prog_lang_dict = Counter({'R':doc_frequency['r'], 'Python':doc_frequency['python'],'Java':doc_frequency['java'], 'C++':doc_frequency['c++'],'Ruby':doc_frequency['ruby'],
                    'Perl':doc_frequency['perl'], 'Matlab':doc_frequency['matlab'],
                    'JavaScript':doc_frequency['javascript'], 'Scala': doc_frequency['scala']})

analysis_tool_dict = Counter({'Excel':doc_frequency['excel'],  'Tableau':doc_frequency['tableau'],
                        'D3.js':doc_frequency['d3.js'], 'SAS':doc_frequency['sas'],
                        'SPSS':doc_frequency['spss'], 'D3':doc_frequency['d3']})  

hadoop_dict = Counter({'Hadoop':doc_frequency['hadoop'], 'MapReduce':doc_frequency['mapreduce'],
                'Spark':doc_frequency['spark'], 'Pig':doc_frequency['pig'],
                'Hive':doc_frequency['hive'], 'Shark':doc_frequency['shark'],
                'Oozie':doc_frequency['oozie'], 'ZooKeeper':doc_frequency['zookeeper'],
                'Flume':doc_frequency['flume'], 'Mahout':doc_frequency['mahout']})

database_dict = Counter({'SQL':doc_frequency['sql'], 'NoSQL':doc_frequency['nosql'],
                    'HBase':doc_frequency['hbase'], 'Cassandra':doc_frequency['cassandra'],
                    'MongoDB':doc_frequency['mongodb']})


overall_total_skills = prog_lang_dict + analysis_tool_dict + hadoop_dict + database_dict # Combine our Counter objects



In [ ]:
total_results

In [ ]:
df

In [13]:
from nltk.corpus import stopwords
from nltk import collocations
import nltk


In [14]:
nltk.word_tokenize(df.description)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\Vivek/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Vivek\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [15]:
first= df.description.apply(lambda t: t.lower() + " ")[1]

In [16]:
second=nltk.word_tokenize(first)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\Vivek/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Vivek\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [17]:
nltk.bigrams(nltk.word_tokenize(first))

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\Vivek/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Vivek\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [18]:
mytext=nltk.word_tokenize("This is my sentance")

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\Vivek/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Vivek\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Vivek\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [ ]:
set(nltk.bigrams(mytext))

In [ ]:
nltk.bigrams("This is my sentance")

In [ ]:
second.dispersion_plot(['Statistics'])

In [ ]:
set(second)

In [ ]:
first

In [ ]:
list(nltk.bigrams(mytext))

In [ ]:
mytext

In [ ]:
nextText = "my sentance is good"

In [ ]:
list(nltk.bigrams(nltk.word_tokenize(nextText)))

In [ ]:
tokens = nltk.word_tokenize(first)
text = nltk.Text(tokens)

In [ ]:
text.dispersion_plot(['intuit'])

In [ ]:
first

In [42]:
df

,city,company,country,date,description,expired,indeedApply,jobkey,jobtitle,latitude,longitude,origin,snippet,source,sponsored,state
0,San Jose,Adobe,US,"Sat, 19 Mar 2016 01:21:01 GMT",Data Scientist - 43275 Description,false,false,60351a8c21c7ea69,Data Scientist,37.337914,-121.89011,san+jose%2C+ca,"Advanced statistical modeling, machine learnin...",Adobe,false,CA
1,Palo Alto,Genesis Global Management Coporation,US,"Fri, 18 Mar 2016 17:09:03 GMT",Must-Haves5+ years data science experienceDegr...,false,true,b96e8049255fb8bc,Data Scientist,37.43956,-122.14286,san+jose%2C+ca,"Experience coding in Python, R, or Clojure to ...",Indeed,false,CA
2,Palo Alto,Insync Consulting Services,US,"Mon, 21 Mar 2016 21:16:01 GMT","As our company's first data scientist, you’ll ...",false,true,c584e9cc43bcb0fa,Data Scientist,37.43956,-122.14286,san+jose%2C+ca,"Data Scientist experience:. Coding in Python, ...",Indeed,false,CA
3,Mountain View,UserTesting,US,"Thu, 24 Mar 2016 03:06:08 GMT",Job Description What makes the difference bet...,false,false,ae4133ed7f8423de,Data Scientist,37.384617,-122.08242,san+jose%2C+ca,A given day could have you doing Machine Learn...,UserTesting,false,CA
4,Cupertino,Reveille technologies,US,"Wed, 16 Mar 2016 15:20:23 GMT",Data Scientist: - Candidate should have a stro...,false,true,4438dc8a975bda84,Data Scientist:,37.32143,-122.02747,san+jose%2C+ca,Candidate should have worked on different algo...,Indeed,false,CA
5,San Jose,TiVo,US,"Sun, 20 Mar 2016 06:55:32 GMT",Support both internal and external client init...,false,false,b976a589270769c2,Data Scientist,37.4001,-121.89531,san+jose%2C+ca,Working with a team of data engineers and data...,TiVo,false,CA
6,San Jose,EBay,US,"Thu, 03 Mar 2016 03:48:17 GMT",Requisition No.: 121517BR Subsidiary: eBay Ma...,false,false,f87400e4ba147c37,Data Scientist,37.337914,-121.89011,san+jose%2C+ca,"The ideal candidate is an independent, solutio...",Ebay,false,CA
7,San Jose,HCL Technologies,US,"Mon, 07 Mar 2016 12:44:55 GMT",Role : FTE ( Full Time Employment)Year of expe...,false,true,9ee83a44bc428657,Data Scientist,37.337914,-121.89011,san+jose%2C+ca,Apply Data Science and Machine Learning to a v...,Indeed,false,CA
8,Sunnyvale,TCS,US,"Wed, 16 Mar 2016 15:20:56 GMT",- Candidate should have a strong background an...,false,true,d36adf11e3e115ea,"Data Scientist - Sunnyvale, CA",37.368134,-122.03297,san+jose%2C+ca,"Strength in Machine Learning, Statistical Mode...",Indeed,false,CA
9,Los Gatos,Netflix,US,"Wed, 16 Mar 2016 05:03:33 GMT","Netflix is seeking an outgoing, curious, inter...",false,false,2ad11d20dd1131bf,Senior Data Scientist - Machine Learning Research,37.225273,-121.97253,san+jose%2C+ca,"As a senior data scientist, you will:. Bring a...",Netflix,false,CA


In [19]:
df

,city,company,country,date,description,expired,indeedApply,jobkey,jobtitle,latitude,longitude,origin,snippet,source,sponsored,state
0,San Jose,IBM,US,"Thu, 03 Mar 2016 04:06:25 GMT",Job Description The Medical Sieve Group in Co...,False,False,f247f5b867a461de,Data Entry Software Engineer,37.337914,-121.890110,san+jose%2C+ca,The Medical Sieve Group in Cognitive Computing...,IBM,False,CA
1,San Jose,Pull Skill Technologies Inc,US,"Tue, 01 Mar 2016 15:19:46 GMT",Position : Data Scientist – ModellerLocation :...,False,True,1f2583f9877e177f,Data Scientist,37.337914,-121.890110,san+jose%2C+ca,Provide technical expertise in statistical ana...,Indeed,False,CA
2,Mountain View,Intuit,US,"Fri, 04 Mar 2016 05:20:37 GMT",Description Intuit QuickBooks is on a mission ...,False,False,702e092b96f66a52,Data Scientist,37.392857,-122.071430,san+jose%2C+ca,"SAS, R, SQL, S-Plus, etc.). Quickly understand...",Intuit,False,CA
3,Menlo Park,Facebook,US,"Wed, 02 Mar 2016 21:21:24 GMT","(Menlo Park, CA) Careers at Oculus A Facebook ...",False,False,200a1bff8841c9b1,"Data Scientist, Analytics (Oculus)",37.453297,-122.181320,san+jose%2C+ca,"Data Scientist, Analytics (Oculus) (Menlo Park...",Facebook,False,CA
4,Cupertino,PullSkill Technologies Inc,US,"Fri, 04 Mar 2016 16:04:33 GMT",Data Scientist with PythonLocation: Cupertino ...,False,True,ada491bfde6f646b,Data Scientist with Python,37.321430,-122.027470,san+jose%2C+ca,Data Scientist with Python*. Requesting a Pyth...,Indeed,False,CA
5,San Jose,EBay,US,"Thu, 03 Mar 2016 03:48:17 GMT",Requisition No.: 121517BR Subsidiary: eBay Ma...,False,False,f87400e4ba147c37,Data Scientist,37.337914,-121.890110,san+jose%2C+ca,"The ideal candidate is an independent, solutio...",Ebay,False,CA
6,San Jose,Paypal,US,"Thu, 03 Mar 2016 08:36:40 GMT",Requisition No.: 29374BR Subsidiary: PayPal ...,False,False,4c1a428bfb229965,Data Scientist,37.337914,-121.890110,san+jose%2C+ca,"Seeks Data Scientist in San Jose, CA:. Collabo...",PayPal,False,CA
7,San Jose,EBay,US,"Fri, 04 Mar 2016 09:52:37 GMT",Requisition No.: 119481BR Subsidiary: eBay Ma...,False,False,198e733636a1b2dc,Senior Data Scientist/Applied Researcher,37.337914,-121.890110,san+jose%2C+ca,Well versed in areas such as applied statistic...,Ebay,False,CA
8,San Jose,Apidel Technologies,US,"Thu, 03 Mar 2016 21:04:00 GMT",Position: Data Scientist / Big Data Engineer –...,False,True,6be0da3a4e19ef4a,Data Scientist / Big Data Engineer – Advance A...,37.337914,-121.890110,san+jose%2C+ca,Data Scientist / Big Data Engineer – Advance A...,Indeed,False,CA
9,Mountain View,Coursera,US,"Fri, 04 Mar 2016 06:36:20 GMT",Discovering the perfect course is the first an...,False,False,369b442fc8485c3f,"Data Scientist, Search and Discovery",37.387398,-122.061120,san+jose%2C+ca,"We're looking for a talented, independent data...",Coursera,False,CA


In [ ]:
#Get a search term
#if you see a search term in a description, return true.
#if you don't, return false.
#For that search term, add a column to the pandas data frame.
#The title of that column is the search term
#The value is true or false!

In [26]:
nltk.word_tokenize(first)

['position',
 ':',
 'data',
 'scientist',
 '–',
 'modellerlocation',
 ':',
 'san',
 'jose',
 ',',
 'caduration',
 ':',
 'fulltime',
 '/',
 'permanent1',
 ')',
 'data',
 'scientist',
 '–*',
 '*modellerfluent',
 'in',
 'python.hadoop',
 ',',
 'java',
 ',',
 'python',
 ',',
 'hive/pig',
 'for',
 'extracting',
 'and',
 'parsing',
 'data.visualization',
 'skills',
 '-',
 'javascript',
 '(',
 'especially',
 'd3.js',
 ')',
 '.the',
 'person',
 'should',
 'have',
 'some',
 'knowledge',
 'of',
 'stats',
 '&',
 'machine',
 'learning.position',
 ':',
 'data',
 'scientist',
 '–',
 'software',
 'engineer',
 '(',
 'swe',
 ')',
 'modellerlocation',
 ':',
 'san',
 'jose',
 ',',
 'caduration',
 ':',
 'fulltime',
 '/',
 'permanent2',
 ')',
 'data',
 'scientist',
 '–',
 'software',
 'engineer',
 '(',
 'swe',
 ')',
 'statistical/data',
 'modeling/data',
 'mining',
 'and',
 'machine',
 'learningproficiency',
 'in',
 'sql',
 'and',
 'dealing',
 'with',
 'petabytes',
 'of',
 'log',
 'files',
 'and',
 'other'

In [11]:
type(first)

str

In [23]:
mytext = nltk.word_tokenize('This is my sentence')

In [24]:
mytext

['This', 'is', 'my', 'sentence']

In [25]:
nltk.download()

showing info http://www.nltk.org/nltk_data/


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Vivek\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\Vivek\Anaconda3\lib\site-packages\nltk\downloader.py", line 1867, in run
    for msg in self.data_server.incr_download(self.items):
  File "C:\Users\Vivek\Anaconda3\lib\site-packages\nltk\downloader.py", line 529, in incr_download
    for msg in self._download_list(info_or_id, download_dir, force):
  File "C:\Users\Vivek\Anaconda3\lib\site-packages\nltk\downloader.py", line 572, in _download_list
    for msg in self.incr_download(item, download_dir, force):
  File "C:\Users\Vivek\Anaconda3\lib\site-packages\nltk\downloader.py", line 543, in incr_download
    for msg in self.incr_download(info.children, download_dir, force):
  File "C:\Users\Vivek\Anaconda3\lib\site-packages\nltk\downloader.py", line 529, in incr_download
    for msg in self._download_list(info_or_id, download_dir, force):
  File "C:\Users\Vive

True

In [27]:
first = df.description[2]

In [28]:
first

"Description Intuit QuickBooks is on a mission to transform small business lending through its QuickBooks Financing Platform. Powerful analytics and a sophisticated understanding of our customers is at the heart of our business. We are looking for a talented data scientist to join our team and help build financing solutions that delight our SMB customers. If you are excited about joining a highly ambitious and talented startup team, then keep reading!  Responsibilities: Possess a wide-latitude in determining objectives and approaches to solution development on mission critical assignments  Go beyond established analytical thinking and problem-solving by applying creativity to unconventional concepts and out-of-the-box solutions  Quickly understand patterns within large quantity of data and to reference key characteristics using visualization techniques  Collaborate with infrastructure architects in assessing and addressing the requirements for more automated, streamlined systems and fo

In [35]:
tokens = nltk.word_tokenize(first)

In [37]:
text = nltk.Text(tokens)

In [38]:
text.collocations()

1-3+ years’; Engineering Mathematics; Theoretical/Computational
Physics; small businesses; related field; business decisions; years’
experience; Proven experience


In [34]:
nltk.word_tokenize(first)

['Description',
 'Intuit',
 'QuickBooks',
 'is',
 'on',
 'a',
 'mission',
 'to',
 'transform',
 'small',
 'business',
 'lending',
 'through',
 'its',
 'QuickBooks',
 'Financing',
 'Platform',
 '.',
 'Powerful',
 'analytics',
 'and',
 'a',
 'sophisticated',
 'understanding',
 'of',
 'our',
 'customers',
 'is',
 'at',
 'the',
 'heart',
 'of',
 'our',
 'business',
 '.',
 'We',
 'are',
 'looking',
 'for',
 'a',
 'talented',
 'data',
 'scientist',
 'to',
 'join',
 'our',
 'team',
 'and',
 'help',
 'build',
 'financing',
 'solutions',
 'that',
 'delight',
 'our',
 'SMB',
 'customers',
 '.',
 'If',
 'you',
 'are',
 'excited',
 'about',
 'joining',
 'a',
 'highly',
 'ambitious',
 'and',
 'talented',
 'startup',
 'team',
 ',',
 'then',
 'keep',
 'reading',
 '!',
 'Responsibilities',
 ':',
 'Possess',
 'a',
 'wide-latitude',
 'in',
 'determining',
 'objectives',
 'and',
 'approaches',
 'to',
 'solution',
 'development',
 'on',
 'mission',
 'critical',
 'assignments',
 'Go',
 'beyond',
 'establis

In [4]:
k = 'Hive'
[w for w in text if k in w]

NameError: name 'text' is not defined

In [1]:
import crab

ImportError: No module named 'crab'

In [3]:
import crab

ImportError: No module named 'crab'